# Neural CSAD

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from psutil import virtual_memory
import tensorflow as tf
plt.style.use('ggplot')
%matplotlib inline

In [2]:
#optimizing memory load
mem = virtual_memory()
size = int(mem.total/100000)
size = 10000
#Chargement des chunks en mémoires
df = pd.read_csv('./project_data/SET1V_01.CSV.gz', compression='gzip',names=['date','out_id', 'in_id', 'calls', 'duration'], parse_dates=['date'],sep=',', quotechar='"', chunksize=size)

In [3]:

#sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=[None, 4])
#test de prédiction de juste les calls
y_ = tf.placeholder(tf.float32, shape=[None, 1])
w = tf.Variable(tf.zeros([4,1]))
b = tf.Variable(tf.zeros([1]))
y = tf.matmul(x,w)+b
euclidian_loss = tf.sqrt(tf.pow(y - y_,2))
loss = tf.reduce_mean(euclidian_loss)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

tf.scalar_summary("loss",loss)
with tf.Session() as sess:
    merged = tf.merge_all_summaries()
    train_writer = tf.train.SummaryWriter('./logs/', sess.graph)
    sess.run(tf.initialize_all_variables())
    i=0
    print('Initialized')
    for chunk in df:
        i +=1
        chunk['day'] = chunk.date.dt.day
        chunk['hour'] = chunk.date.dt.hour
        chunk1 = np.array(chunk[['day','hour','out_id','in_id']])
        chunk2 = np.array(chunk['calls'])
        summary, _ , lo  = sess.run([merged,train_step,loss], feed_dict={x: chunk1 ,y_: chunk2[np.newaxis, :].T})
        train_writer.add_summary(summary,i)
        if ((i % 100) == 0 ):
            print('session ' + str(i) + ' -- loss ' + str(lo))
            #sys.stdout.write('\r >> session %s -- loss : %s' % (i,lo))
            #sys.stdout.flush()
        if (i>=60000):
            break


Initialized
session 100 -- loss 352883.0
session 200 -- loss 388614.0
session 300 -- loss 81466.8
session 400 -- loss 324219.0
session 500 -- loss 105502.0
session 600 -- loss 778836.0
session 700 -- loss 98084.5
session 800 -- loss 37547.5
session 900 -- loss 129176.0
session 1000 -- loss 71024.2
session 1100 -- loss 18564.3
session 1200 -- loss 43135.4
session 1300 -- loss 149403.0
session 1400 -- loss 561748.0
session 1500 -- loss 174484.0
session 1600 -- loss 36119.1
session 1700 -- loss 129195.0
session 1800 -- loss 139468.0
session 1900 -- loss 149799.0
session 2000 -- loss 79362.6
session 2100 -- loss 43964.4
session 2200 -- loss 40800.9
session 2300 -- loss 511159.0
session 2400 -- loss 477073.0
session 2500 -- loss 45939.6
session 2600 -- loss 589078.0
session 2700 -- loss 107940.0
session 2800 -- loss 41212.2
session 2900 -- loss 450987.0
session 3000 -- loss 46874.4
session 3100 -- loss 697557.0
session 3200 -- loss 134570.0


KeyboardInterrupt: 

In [7]:
 # Test trained model
chunk = next(df)
chunk['day'] = chunk.date.dt.day
chunk['hour'] = chunk.date.dt.hour
chunk1 = np.array(chunk[['day','hour','out_id','in_id']])
chunk2 = np.array(chunk['calls'])
#TODO changer la métrique
#correct_prediction = (tf.r)

accuracy = tf.reduce_mean(tf.cast(euclidian_loss, tf.float32))

In [ ]:
print(sess.run(accuracy, feed_dict={x: chunk1, y_: chunk2[np.newaxis, :].T}))